# Ekstrakcja koreferencji przez multiservice

Dokumnet opisuje przetwarzanie plików przez usługę IPIPAN Multiservice w celu wykrywania koreferencji.

Przygotowano do analizy zbiór 3035 zdań w kontekście 2 zdań przed i 1 po czyli 4 zadnia na plik TXT.

Użyto narzędzia do analizy tekstu w następującym pipeline:

1. Pantera - segmentacja i analiza morfo-syntaktyczna
2. Spejd - płytkie parsowanie i rozpoznawanie wyrazów i grup wyrazów
3. Nerf - rozpoznawanie nazw własnych
4. MentionDetector - detekcja kandydatów do koreferencji
5. Bartek - rozpoznawanie koreferencji

### Bartek

Nazwa pochodzi z systemu [BART](http://www.bart-coref.org/) - Beautiful Anaphora Resolution Toolkit.

System został wytrenowany na [Polskim Korpusie Koreferencyjnym](http://zil.ipipan.waw.pl/PolishCoreferenceCorpus?action=AttachFile&do=view&target=PCC_README_PL.pdf) zawierającym:
* 1773 tekstów „krótkich”, czyli o długości 250-350 segmentów, będących wybranymi fragmentami dłuższych dokumentów (wybranymi jako pewna liczba pełnych, kolejnych akapitów z pojedynczego źródła).
* 21 tekstów „długich” – pełnych tekstów o długości 1000-4000 segmentów.

Rodzaje tekstów to: Dzienniki, Pozostałe periodyki, Literatura piękna, Literatura faktu, Typ informacyjno-poradnikowy, Mówione konwersacyjne, Internetowe nieinteraktywne (statyczne strony), Internetowe interaktywne (blogi, fora, usenet), Inne teksty pisane, Mówione medialne, Quasi-mówione (protokoły sesji parlamentu), Typ naukowo-dydaktyczny, Książki publicystyczne, Książka niebeletrystyczna nieklasyfikowana. Szczgóły są pod linkiem wyżej.

Segmenty to jednostki o roli podobnej do słów, pozwalające na oddanie morfoskładniowych właściwości w języku  polskim (oraz pewnych decyzji podjętych arbitralnie); np. aglutynant (długo+śmy), partykuła -że (znasz+że) itp. są uznawane za osobne segmenty.

## Wczytywanie danych

W repozytorioum skomporesowałem wszystkie wyniki, bo w wersji rozpakowanej jest kilka tysięcy plików ważacych ponad pół GB. Jeśli nie jest to już zrobione, warto wykonać następujące:

In [ ]:
!unzip paragraphs.zip

In [1]:
import json

Wczytajmy jeden dowolny plik:

In [2]:
with open('paragraphs/10.json') as f:
    data=json.load(f)

## Opis struktury danych

#### Każdy plik zawiera w najwyżsym poziomie

In [3]:
data.keys()

dict_keys(['annotationDetails', 'annotationHeaders', 'coreferences', 'paragraphs', 'summary', 'textHeader'])

#### Nieistotne pola

* annotationDetails - globalne parametry
* annotationHeaders - lista użytych narzędzi
* textHeader - opis danych, czasu przetwarzania, itp
* summary - puste

In [4]:
print(data['annotationDetails'])
print(data['annotationHeaders'])
print(data['textHeader'])
print(data['summary'])

{'hasMorphosyntaxDisambiguated': True, 'hasMorphosyntaxPartiallyDisambiguated': True, 'hasSegmentsDisambiguated': True}
{'0': {'distributor': 'Pantera', 'id': '', 'processingDuration': 798, 'publicationTime': 1553556031041, 'retrievedFrom': '', 'sourceDescText': '', 'title': ''}, '1': {'distributor': 'Pantera; modified by Spejd', 'id': '', 'processingDuration': 2786, 'publicationTime': 1553556031041, 'retrievedFrom': '', 'sourceDescText': '', 'title': ''}, '2': {'distributor': 'Spejd', 'id': '', 'processingDuration': 1988, 'publicationTime': 1553556031041, 'retrievedFrom': '', 'sourceDescText': '', 'title': ''}, '3': {'distributor': 'Spejd', 'id': '', 'processingDuration': 1988, 'publicationTime': 1553556031041, 'retrievedFrom': '', 'sourceDescText': '', 'title': ''}, '4': {'distributor': 'Nerf', 'id': '', 'processingDuration': 95, 'publicationTime': 1553556031041, 'retrievedFrom': '', 'sourceDescText': '', 'title': ''}, '7': {'distributor': 'MentionDetector', 'id': None, 'processingDu

### Koreferencje

Lista koreferencji zawiera:
* dominat - opis w postaci ortograficznej
* id - ID korerferencji
* mentionIds - wystąpienia korereferencji w tekście
* sourceMentionId - zawsze puste
* type - zawsze ident (czyli "identity")

In [5]:
data['coreferences'][3]

{'dominant': 'świnka',
 'id': 'c-3',
 'mentionIds': ['m-3', 'm-5'],
 'sourceMentionId': None,
 'type': 'ident'}

### Paragraf

#### Podstawowe informacje

In [6]:
print(f'Ilość akapitów (zawsze 1): {len(data["paragraphs"])}')
par=data['paragraphs'][0]
print(f'Akapit zawiera: {list(par.keys())}')
print(f'Ilość zdań w akapicie (powinno być 4): {len(par["sentences"])}')
print(f'Każde zdanie zawiera: {list(par["sentences"][0].keys())}')

Ilość akapitów (zawsze 1): 1
Akapit zawiera: ['id', 'sentences', 'text']
Ilość zdań w akapicie (powinno być 4): 1
Każde zdanie zawiera: ['dependencyParse', 'groups', 'id', 'mentions', 'names', 'rejectedTokens', 'sentimentTags', 'tokens', 'words']


#### Tekst całego akapitu

In [7]:
par['text']

' szczepionka przeciw odrze, różyczce i śwince: zawiera: żywe wirusy odry, świnki, różyczki hodowane na zarodkach kurzych, neomycynę (toksyczny antybiotyk), żelatynę (częsty alergen), sorbitol.  Wydaje się nienaturalnym łączenie w jednej szczepionce tych trzech chorób, które nigdy nie występują jednocześnie.  Odporność nabyta po ich zaszczepieniu trwa krócej niż odporność nabyta po przebyciu choroby.  W dodatku szczepionki te są bardzo nietrwałe i, na przykład, po wystawieniu na słońce tracą wartość.'

#### Puste nieistotne pola

In [8]:
sent=par['sentences'][0]
print(sent["dependencyParse"])
print(sent["names"])
print(sent["rejectedTokens"])
print(sent["sentimentTags"])

[]
[]
[]
[]


#### Istotne informacje

Nieposte pola to:
* groups - znaczeniowe grupy wyrazów
* mentions - lista wystąpień koreferencji ze wskaźnikami do wyrazów
* tokens - tokeny (text -> Pantera -> tokeny)
* words - słowa (tokeny -> Spejd -> słowa/grupy)

## Wyszukiwanie koreferencji

Algorytm:
1. tworzymy mapę mention: mention ID -> opis koreferencji
    a. zapisujemy tylko te koreferencje które mają więcej niż 1 wystąpienie
2. tworzymy mapę segmentów: segment ID -> zapis ortograficzny
3. dla poszczególnych zdań i poszczególnych wystąpień koreferencji,
    jeśli dane wystąpienie jest z koreferencji w którym jest więcej wystąpień
    wypisz następujące linie:

```
_główny segment_ pozostałe segmenty -> opis koreferencji
```

In [9]:
with open('paragraphs/10.json') as f:
    data=json.load(f)

par=data['paragraphs'][0]
print('Tekst:\n')
print(par['text'])
print('\nKoreferencje:\n')

mention={}
for c in data['coreferences']:
    if len(c['mentionIds'])>1:
        for m in c['mentionIds']:
            mention[m]=c['dominant']
        
segs={}
for sent in par['sentences']:
    for t in sent['tokens']:
        segs[t['id']]=t['orth']

for sent_num,sent in enumerate(par['sentences']):
    for m in sent['mentions']:
        if m['id'] in mention:
            head=m['headIds'][0]
            w=[]        
            for c in m['childIds']:
                if c==head:
                    w.append('_'+segs[c]+'_')
                else:
                    w.append(segs[c])
            print(f'{" ".join(w)} -> {mention[m["id"]]}')

Tekst:

 szczepionka przeciw odrze, różyczce i śwince: zawiera: żywe wirusy odry, świnki, różyczki hodowane na zarodkach kurzych, neomycynę (toksyczny antybiotyk), żelatynę (częsty alergen), sorbitol.  Wydaje się nienaturalnym łączenie w jednej szczepionce tych trzech chorób, które nigdy nie występują jednocześnie.  Odporność nabyta po ich zaszczepieniu trwa krócej niż odporność nabyta po przebyciu choroby.  W dodatku szczepionki te są bardzo nietrwałe i, na przykład, po wystawieniu na słońce tracą wartość.

Koreferencje:

_odrze_ -> Odra
_śwince_ -> świnka
_odry_ -> Odra
_świnki_ -> świnka
_Odporność_ nabyta -> odporność nabyć
_odporność_ nabyta -> odporność nabyć


### Szukanie koreferencji wystąpienia wyrazu drugiego zdania z wystąpieniem poza tym zdaniem

W tej chwili nie działa bo się popsuło oddzielanie zdań.

In [ ]:
with open('paragraphs/5.json') as f:
    data=json.load(f)

par=data['paragraphs'][0]
print(par['text'])

mention={}
for c in data['coreferences']:
    if len(c['mentionIds'])>1:
        mlist=set()
        for m in c['mentionIds']:
            mlist.add(m)
            mention[m]=mlist
segs={}
for sent in par['sentences']:
    for t in sent['tokens']:
        segs[t['id']]=t['orth']
        
orth={}
for sent_num,sent in enumerate(par['sentences']):
    for m in sent['mentions']:
        if m['id'] in mention:
            head=m['headIds'][0]
            w=[]        
            for c in m['childIds']:
                if c==head:
                    w.append('_'+segs[c]+'_')
                else:
                    w.append(segs[c])
            orth[m['id']]=(' '.join(w),sent_num)

for m in par['sentences'][2]['mentions']:
    if m['id'] in mention:
        for n in mention[m['id']]:
            if orth[n][1]!=2:
                print(f'{orth[m["id"]][0]} -> {orth[n][0]}')
        
    